In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import math
from dateutil.relativedelta import relativedelta
from sklearn import preprocessing

In [20]:
from utils import model_utils
from utils import db_utils
from utils import iefp_data_utils
from utils import pandas_utils
from utils import feature_utils

In [4]:
pd.options.mode.chained_assignment = None

#Function definitions

In [5]:
#Demographic feature functions
def apps_cancelled_within_n_months(x,n=12):
    return x[(x['movement_type'].isin(['cancellation'])) & (x['months_after_app'] < n)]['application_id'].unique()

def apps_placed_within_n_months(x,n=12):
    return x[(x['movement_result'].isin(['ADMITIDO / COLOCADO'])) & (x['months_after_app'] < n)]['application_id'].unique()

#Add column to show how many months after the application the movement occurred

def monthdelta(d1, d2):
    delta = 0
    while True:
        mdays = monthrange(d1.year, d1.month)[1]
        d1 += timedelta(days=mdays)
        if d1 <= d2:
            delta += 1
        else:
            break
    return delta

def difftime_in_months(timeA,timeB):
    return (timeA-timeB)/np.timedelta64(1, 'M')

def get_ltu_label(apps,movs):
    app_date_dict = dict(zip(apps["table_index"],apps["app_start_date"]))
    movs["app_start_date"] = movs["application_id"].map(app_date_dict)
    movs["months_after_app"] = difftime_in_months(movs["movement_event_date"],movs["app_start_date"])
    
    print "Generating LTU/Non-LTU labels"
    cancelled_12mo = apps_cancelled_within_n_months(movs)
    placed_12mo = apps_placed_within_n_months(movs)
    
    non_ltu_apps = np.unique(np.append(cancelled_12mo,placed_12mo))
    apps['ltu'] = np.logical_not(apps['table_index'].isin(non_ltu_apps))
    last_data_date = max(apps['app_start_date'])
    apps.loc[difftime_in_months(last_data_date,apps['app_start_date']) < 12,'ltu'] = False
    
    return apps

def findSeason(x):
    month=x.month
    if month in [12,1,2]:
        return "Winter"
    if month in [3,4,5]:
        return "Spring"
    if month in [6,7,8]:
        return "Summer"
    if month in [9,10,11]:
        return "Autumn"

def isPortuguese(x):
    if x=="PT":
        return "PT"
    else:
        return "OT"

def lookingForFirstJob(x):
    if "NOVO" in x:
        return "first_job"
    else:
        return "not_first_job"

def createEducationBuckets(x):
    HS=["12"]
    #SL means  able to write, but no school degree,NS means not able to write
    NR=["NS"]
    U6=["SL","04","06"]
    U11=["09","11"]
    
    #The rest is different kinds of higher degree
    if x in HS:
        return "HS"
    if x in NR:
        return "NR"
    if x in U6:
        return "U6"
    if x in U11:
        return "U11"
    else:
        return "MHS"

def civilStatus(x):
    if x=="S":
        return "S"
    if x=="C":
        return "M"
    else :
        return "O"

def ageBucket(x):
    if x<30:
        return "age<30"
    elif x<50:
        return "30<age<50"
    else:
        return "age>50"

def findMainBucket(profession):
    profDict={0:"Miliatary",1:"Managment",2:"Higer level professionals",3:"Lower level professionals",4:"Office jobs",5:"Sales",
              6:"Fising and farming",7:"Craft and trade",8:"Factory work",9:"Elementary"}
    if pd.isnull(profession):
        return "Unknown"
    else:
        professionInt=int(str(profession)[0:1])
        return profDict[professionInt]
    
def findMainEducation(education):
    eduDict={0:"general",1:"Education",2:"ArtsAndHumanities",3:"SocialScienceTradeAndLaw",4:"STEM",5:"EngineeringAndConstruction",6:"Agriculture",7:"HealthAndSocialProtection",8:"Service",9:"Unkown"}
    if pd.isnull(education):
        return eduDict[9]
    else:
        professionInt=int(str(education)[0:1])
        return eduDict[professionInt]

def preferredJobRegion(letter):
    if letter == "C":
        return "NearJobCenter"
    elif letter == "R":
        return "GreaterLisbonArea"
    elif letter == "N":
        return "Available"
    elif pd.isnull(letter):
        return "Unknown"
    else:
        return "Error"

def employmentPlan(letter):
    if letter == "S":
        return "exists"
    elif letter =="N":
        return "no plan"
    else:
        return "Error"
    
def applicationOrigin(origin):
    if origin == "SIGAE":
        return "SIGAE"
    elif origin == "LSE":
        return "LSE"
    elif pd.isnull(origin):
        return "unknown"
    else:
        return "Error"
    
def intendedRegime(regime):
    if regime == "COMPLETO":
        return "Full-time"
    elif regime == "PARCIAL":
        return "Part-time"
    else:
        return "Error"

def dependentsBucket(n):
    if n == 0:
        return "0"
    elif n == 1:
        return "1"
    elif n == 2:
        return "2"
    elif n == 3:
        return "3"
    elif pd.isnull(n):
        return "Unknown"
    else:
        return "4+"
    
def experienceBuckets(n):
    if pd.isnull(n):
        return "Unknown"
    elif n == 0:
        return "0"
    elif n<24:
        return "<2yr"
    elif n<60:
        return "<5yr"
    elif n<120:
        return "<10yr"
    elif n<240:
        return "<20yr"
    else:
        return "20+yr"

In [6]:
#Dynamic variable functions

def generate_cumsum(movs):
    movs_count = movs.drop(['movement_subtype','movement_result','movement_index','ute_id'], axis=1)
    movs_count['movements'] = 1
    movs_count = pd.get_dummies(movs_count, columns=['movement_type'])
    movs_cumsum = movs_count.groupby(['application_id','movement_event_date']).sum().groupby(level=[0]).cumsum()
    movs_cumsum.reset_index(level=[0,1],inplace=True)
    column_names = {'movement_type_application':'apps_so_far', 'movement_type_cancellation':'cancellations_so_far',\
                   'movement_type_category_change':"cat_changes_so_far", 'movement_type_convocation':'convocations_so_far',\
                   'movement_type_intervention':'interventions_so_far','movement_type_interview':'interviews_so_far',}
    movs_cumsum.rename(columns=column_names,inplace=True)
    if 'cancellations_so_far' in movs_cumsum.columns:
        #Remove because test set should not have cancellations 
        movs_cumsum.drop(['cancellations_so_far'],axis=1,inplace=True)

    return movs_cumsum

def generate_movs_so_far(extended_data, movs):
    movs_cumsum = generate_cumsum(movs)
    
    extended_trimmed = extended_data.drop(['app_start_date','ltu'], axis=1)
    extended_trimmed['entry_type'] = 'observation'
    movs_cumsum['entry_type'] = 'mov_date'
    
    #combine the extended data (has observation dates) with movements (has cumsum data)
    movs_so_far = pd.concat([extended_trimmed, movs_cumsum])
    movs_so_far['sort_date'] = movs_so_far['ref_date'].fillna(movs_so_far['movement_event_date'])
    
    #sort by date and forward fill (so that observation has cumsums from the closest movement_event_date before it)
    movs_so_far.sort_values(['application_id','sort_date','entry_type'], inplace=True)
    movs_so_far.drop(['movement_event_date','ref_date'], axis=1,inplace=True)
    movs_so_far.fillna(method='ffill', inplace=True)
    
    return movs_so_far[movs_so_far['entry_type']=="observation"]


In [7]:
# SYSTEM FEATURES FUNCTIONS
def generate_system_info(apps):
    system_info = apps.copy()
    system_info['app_year']=system_info['app_start_date'].apply(lambda x: x.year)
    system_info['app_month']=system_info['app_start_date'].apply(lambda x: x.month)
    system_info = system_info.groupby(['app_year','app_month']).anomes.count()
    return system_info

In [8]:
def add_demographics(feature_matrix, apps):
    #Create DF with the demographic features per application_id
    demo_df = apps[["table_index"]]
    
    demo_df["age"] = apps["ute_idade"] #redo age variable to make dyanamic?
    demo_df["gender"] = apps["sexo"]
    demo_df["is_re_registriation"] = apps["candidatura_rinsc"]
    demo_df["soc_ben"] = apps["sub_rsi"]
    #demo_df["age2"]=demo_df["age"]**2
    demo_df["age"]=preprocessing.scale(demo_df["age"].astype(float),copy=False)
    #demo_df["age2"]=preprocessing.scale(demo_df["age2"].astype(float),copy=False)
    demo_df["season"]=apps["app_start_date"].apply(findSeason)
    demo_df["nationality"]=apps["cnacionalidade"].apply(isPortuguese)
    demo_df["education"]=apps["chabilitacao_escolar"].apply(createEducationBuckets) 
    demo_df["first_job"]=apps["dcategoria"].apply(lookingForFirstJob)
    demo_df["time_since_exit"]=(pd.to_datetime(apps["candidatura_data"])-pd.to_datetime(apps["reinscricao_ult_saida_data"])).dt.days
    demo_df["time_since_exit"].fillna(500,inplace=True)
    demo_df["time_since_exit"]=pd.to_numeric(demo_df["time_since_exit"])
    demo_df["time_since_exit"]=preprocessing.scale(demo_df["time_since_exit"], copy=False)
    demo_df["is_disabled"]=apps["cdeficiencia"].apply(lambda x: "N" if x==0 else "S")
    demo_df["civil_status"]=apps["ute_estado_civil"].apply(civilStatus)
    demo_df["has_course_area"]=apps["darea_curso_tabela_em_activo"]
    demo_df["has_course_area"].fillna(0,inplace=True)
    demo_df["has_course_area"]=demo_df["has_course_area"].apply(lambda x: "S" if x==0 else "N")
    demo_df["has_prof_area"]=apps["darea_formacao_tabela_em_activo"]
    demo_df["has_prof_area"].fillna(0,inplace=True)
    demo_df["has_prof_area"]=demo_df["has_prof_area"].apply(lambda x: "S" if x==0 else "N")
    demo_df["age_category"]=demo_df["age"].apply(ageBucket)
    demo_df["number_dependents"]=apps["ute_nr_pessoas_cargo"].astype(float).fillna(0)
    demo_df["number_dependents"]=preprocessing.scale(demo_df["number_dependents"], copy=False)
    demo_df["experience_intended_prof"]=apps["candidatura_prof_pret_tempo_pratica"].astype(float).fillna(0)
    demo_df["experience_intended_prof"]=preprocessing.scale(demo_df["experience_intended_prof"], copy=False)
    demo_df["experience_prev_prof"]=apps["sit_anterior_prof_tempo_pratica"].astype(float).fillna(0)
    demo_df["experience_prev_prof"]=preprocessing.scale(demo_df["experience_prev_prof"], copy=False) 
    demo_df["intended_prof_1"]=apps["cnp_pretendida"].apply(findMainBucket)
    demo_df["intended_prof_2"]=apps["cpp_pretendida"].apply(findMainBucket)
    demo_df["intended_prof"]=np.where(pd.isnull(demo_df["intended_prof_1"]),demo_df["intended_prof_2"],demo_df["intended_prof_1"])
    del demo_df["intended_prof_1"]
    del demo_df["intended_prof_2"]  
    demo_df["previous_prof_1"]=apps["cnp_anterior"].apply(findMainBucket)
    demo_df["previous_prof_2"]=apps["cpp_anterior"].apply(findMainBucket)
    demo_df["previous_prof"]=np.where(pd.isnull(demo_df["previous_prof_1"]),demo_df["previous_prof_2"],demo_df["previous_prof_1"])
    del demo_df["previous_prof_1"]
    del demo_df["previous_prof_2"]   
    demo_df["training_area"]= apps["carea_formacao_tabela_em_activo"].apply(findMainEducation)   
    #demo_df['app_start_date'] = pd.to_datetime(apps['app_start_date'])  #already in the extended data
    demo_df['preferred_location']=apps['candidatura_local_trabalho'].apply(preferredJobRegion)
    demo_df['app_year'] = apps['app_start_date'].apply(lambda x: x.year)
    demo_df['app_month'] = apps['app_start_date'].apply(lambda x: x.month)
    demo_df['employment_plan'] = apps['ute_plano_emprego'].apply(employmentPlan)
    demo_df['prev_employment_plan'] = apps['ute_plano_emprego_anterior'].apply(employmentPlan)
    demo_df['app_origin'] = apps['candidatura_origem'].apply(applicationOrigin)
    demo_df['intended_regime'] = apps['dregime_contrato_pretendido'].apply(intendedRegime)
    demo_df['dependents_bucket'] = apps['ute_nr_pessoas_cargo'].astype(float).apply(dependentsBucket)
    demo_df['exp_intended_prof_buckets'] = apps['candidatura_prof_pret_tempo_pratica'].astype(float).apply(experienceBuckets)
    demo_df['exp_previous_prof_buckets'] = apps['sit_anterior_prof_tempo_pratica'].astype(float).apply(experienceBuckets)
    
    #map to feature matrix using application ID
    feature_matrix = pd.merge(feature_matrix, demo_df, how='left', left_on='application_id', right_on='table_index')
    feature_matrix.drop('table_index',axis=1,inplace=True) 
    return feature_matrix

In [9]:
def add_dynamic_features(feature_matrix, extended_data, movs):
    #generate movs_so_far variables
    movs_so_far = generate_movs_so_far(extended_data, movs)
    feature_matrix = pd.merge(feature_matrix, movs_so_far, how='left', left_on = ['application_id', 'ref_date'],\
                             right_on=['application_id','sort_date'])
    feature_matrix.drop(['sort_date', 'entry_type'], inplace=True, axis=1)
    #slow but precise months since app
    #feature_matrix['months_so_far'] = feature_matrix.apply(lambda x: monthsdiff(x['app_start_date'],x['ref_date']),axis=1)
    
    #faster months since app
    feature_matrix['months_so_far'] = (feature_matrix['ref_date']-feature_matrix['app_start_date']).apply(lambda x: x.days/30)
    return feature_matrix

In [10]:
def add_system_features(feature_matrix,system_info):
    feature_matrix['total_apps_in_month'] = feature_matrix['ref_date'].apply(lambda x: system_info[x.year, x.month])
    return feature_matrix

In [11]:
def add_historical_features(feature_matrix, historical_info):
    pass

In [12]:
#Generate matrix function that takes apps and movs tables as inputs
def generate_matrix(extended_data, apps_cropped, movs_cropped, feature_set_list,system_info):
    #map movements to application start date
    #app_date_dict = dict(zip(extended_data["table_index"],extended_data["app_start_date"]))
    #movs["app_start_date"] = movs["application_id"].map(app_date_dict)

    print "Generating matrix base" #generate basic matrix including appID, application date, months so far, and label
    feature_matrix = extended_data.copy()
    
    if "demographics" in feature_set_list:
        print "Adding demographic features"
        feature_matrix = add_demographics(feature_matrix, apps_cropped)
    
    if "dynamic" in feature_set_list:
        print "Adding dynamic features"
        feature_matrix = add_dynamic_features(feature_matrix, extended_data, movs_cropped)
        
    if "system" in feature_set_list:
        print "Adding system features"
        feature_matrix = add_system_features(feature_matrix,system_info)
    
    #Removing Irrelevant features and create dummies for categorical variables
    irrelevant_features = ['app_start_date']
    feature_matrix.drop(irrelevant_features,1,inplace=True)
    
    #Take observation identification variables and set as index
    feature_matrix.set_index(['application_id','ref_date'], inplace=True)
    feature_matrix = pd.get_dummies(feature_matrix,drop_first=True)
    
    return feature_matrix

#Variable Definitions

In [13]:
split_type = "action"
action_date = pd.to_datetime("2015-04-30")
train_timedelta = pd.Timedelta("730D")
test_window_size = pd.Timedelta("365D")
train_st_date = action_date - train_timedelta
ltu_length = 12

# split_type = "action"
# action_date = pd.to_datetime("2016-04-30")
# train_timedelta = pd.Timedelta('730D')
# test_window_size = pd.Timedelta('365D')

# train_st_date = action_date - train_timedelta
# ltu_length = 12

#Running Pipeline

In [14]:
#Reading Data from DB
conn = db_utils.connect_to_db()
apps, movs = iefp_data_utils.get_clean_data(conn)

In [21]:
#generate non-changing system and historical dataframes
system_info = feature_utils.generate_system_info(apps)

In [15]:
#Splitting train/test datasets
from utils import train_test_utils
train_apps,test_apps,train_movs,test_movs = train_test_utils.split_train_test_apps(apps,movs,action_date,train_st_date,ltu_length)

In [16]:
#Generate LTU Label
train_labels = train_test_utils.get_ltu_label_on_date(train_apps,movs,action_date,ltu_length)
test_labels = train_test_utils.get_ltu_label_on_date(test_apps,movs,action_date + test_window_size,ltu_length)

Generating LTU/Non-LTU labels
Generating LTU/Non-LTU labels


In [17]:
#Extend data
extended_train = train_test_utils.extend_data(train_apps,movs,train_labels,action_date,pd.Timedelta('30D'))
extended_test = train_test_utils.extend_data(test_apps,movs,test_labels,action_date,None)

In [22]:
#Generate Features
train_matrix = generate_matrix(extended_train,train_apps,train_movs,['demographics','dynamic'],system_info)
test_matrix = generate_matrix(extended_test,test_apps,test_movs,['demographics','dynamic'],system_info)

Generating matrix base
Adding demographic features
Adding dynamic features
Generating matrix base
Adding demographic features
Adding dynamic features


In [ ]:
set(train_matrix.columns.tolist()).difference(test_matrix.columns.tolist())
#train_matrix.columns.tolist().set(test_matrix.columns.tolist())

In [ ]:
test_movs.head()

In [ ]:
train_movs.head()

In [ ]:
print len(train_matrix.columns)
print len(test_matrix.columns)

In [ ]:
def train_model(model_type,train_matrix,model_params):
    y = train_matrix.ltu.astype(float)
    X = train_matrix.drop(['ltu'], 1)
    if model_type=="lg":
        logit = sm.Logit(y,X.astype(float))
        model = logit.fit(maxiter=1000)
    else:
        model = None
        model = model_utils.get_model(model_type)
        #Define Model input/output
        print len(X.columns)
        model.fit(X=X, y=y)
    return model

def test_model(model_type,model,test_matrix,model_id):
    if model_type=="lg":
        pred = model.predict(test_matrix.drop(['ltu'],1))
        model_results = pd.Series(index=test_matrix.index, data=pred)
    else:
        print len(test_matrix.drop(['ltu'],1).columns)
        pred = model.predict_proba(test_matrix.drop(['ltu'],1))
        model_results = pd.Series(index=test_matrix.index, data=pred[:,1])
    labels = pd.DataFrame(test_matrix['ltu'])
    all_model_results = pd.concat([labels,pd.DataFrame(data=model_results,columns=[model_id])], axis=1)
    return all_model_results

In [ ]:
model_results.head()

In [23]:
train_matrix.head()

ltu       age  time_since_exit  \
application_id ref_date                                       
74067          2013-09-25  False -0.891789        -0.550967   
               2013-08-25  False -0.891789        -0.550967   
               2013-07-25  False -0.891789        -0.550967   
               2013-06-25  False -0.891789        -0.550967   
               2013-05-25  False -0.891789        -0.550967   

                           number_dependents  experience_intended_prof  \
application_id ref_date                                                  
74067          2013-09-25          -0.548568                 -0.657717   
               2013-08-25          -0.548568                 -0.657717   
               2013-07-25          -0.548568                 -0.657717   
               2013-06-25          -0.548568                 -0.657717   
               2013-05-25          -0.548568                 -0.657717   

                           experience_prev_prof  app_year  app_month  \
application_id ref_date                                                
74067          2013-09-25             -0.664818      2013          4   
               2013-08-25             -0.664818      2013          4   
               2013-07-25             -0.664818      2013          4   
               2013-06-25             -0.664818      2013          4   
               2013-05-25             -0.664818      2013          4   

                           apps_so_far  cat_changes_so_far  \
application_id ref_date                                      
74067          2013-09-25          1.0                 0.0   
               2013-08-25          1.0                 0.0   
               2013-07-25          1.0                 0.0   
               2013-06-25          1.0                 0.0   
               2013-05-25          1.0                 0.0   

                                         ...                  \
application_id ref_date                  ...                   
74067          2013-09-25                ...                   
               2013-08-25                ...                   
               2013-07-25                ...                   
               2013-06-25                ...                   
               2013-05-25                ...                   

                           exp_intended_prof_buckets_<10yr  \
application_id ref_date                                      
74067          2013-09-25                                0   
               2013-08-25                                0   
               2013-07-25                                0   
               2013-06-25                                0   
               2013-05-25                                0   

                           exp_intended_prof_buckets_<20yr  \
application_id ref_date                                      
74067          2013-09-25                                0   
               2013-08-25                                0   
               2013-07-25                                0   
               2013-06-25                                0   
               2013-05-25                                0   

                           exp_intended_prof_buckets_<2yr  \
application_id ref_date                                     
74067          2013-09-25                               1   
               2013-08-25                               1   
               2013-07-25                               1   
               2013-06-25                               1   
               2013-05-25                               1   

                           exp_intended_prof_buckets_<5yr  \
application_id ref_date                                     
74067          2013-09-25                               0   
               2013-08-25                               0   
               2013-07-25                               0   
               2013-06-25                               0   
               

In [30]:
train_matrix = train_matrix.reset_index()

In [36]:
historical_dict = dict(zip(pd.Series([74067]),pd.Series([7])))

In [39]:
train_matrix.head()

,application_id,ref_date,ltu,age,time_since_exit,number_dependents,experience_intended_prof,experience_prev_prof,app_year,app_month,...,exp_intended_prof_buckets_<10yr,exp_intended_prof_buckets_<20yr,exp_intended_prof_buckets_<2yr,exp_intended_prof_buckets_<5yr,exp_previous_prof_buckets_20+yr,exp_previous_prof_buckets_<10yr,exp_previous_prof_buckets_<20yr,exp_previous_prof_buckets_<2yr,exp_previous_prof_buckets_<5yr,exp_previous_prof_buckets_Unknown
0,74067,2013-09-25,False,-0.891789,-0.550967,-0.548568,-0.657717,-0.664818,2013,4,...,0,0,1,0,0,0,0,1,0,0
1,74067,2013-08-25,False,-0.891789,-0.550967,-0.548568,-0.657717,-0.664818,2013,4,...,0,0,1,0,0,0,0,1,0,0
2,74067,2013-07-25,False,-0.891789,-0.550967,-0.548568,-0.657717,-0.664818,2013,4,...,0,0,1,0,0,0,0,1,0,0
3,74067,2013-06-25,False,-0.891789,-0.550967,-0.548568,-0.657717,-0.664818,2013,4,...,0,0,1,0,0,0,0,1,0,0
4,74067,2013-05-25,False,-0.891789,-0.550967,-0.548568,-0.657717,-0.664818,2013,4,...,0,0,1,0,0,0,0,1,0,0


In [38]:
train_matrix[train_matrix['application_id']==74067]['application_id'].map(lambda x: historical_dict[x])

0    7
1    7
2    7
3    7
4    7
Name: application_id, dtype: int64

In [40]:
train_matrix.set_index(['application_id', 'ref_date'])

ltu       age  time_since_exit  \
application_id ref_date                                       
74067          2013-09-25  False -0.891789        -0.550967   
               2013-08-25  False -0.891789        -0.550967   
               2013-07-25  False -0.891789        -0.550967   
               2013-06-25  False -0.891789        -0.550967   
               2013-05-25  False -0.891789        -0.550967   
74084          2013-07-25  False -1.224848        -0.177430   
               2013-06-25  False -1.224848        -0.177430   
               2013-05-25  False -1.224848        -0.177430   
74079          2013-08-01  False  0.856772        -0.177430   
               2013-07-01  False  0.856772        -0.177430   
               2013-06-01  False  0.856772        -0.177430   
               2013-05-01  False  0.856772        -0.177430   
74065          2013-08-20  False  0.273919        -0.177430   
               2013-07-20  False  0.273919        -0.177430   
               2013-06-20  False  0.273919        -0.177430   
               2013-05-20  False  0.273919        -0.177430   
74078          2013-08-29  False  0.357183        -0.177430   
               2013-07-29  False  0.357183        -0.177430   
               2013-06-29  False  0.357183        -0.177430   
               2013-05-29  False  0.357183        -0.177430   
74071          2013-05-28  False -1.224848        -0.177430   
74069          2014-01-27  False  0.440448        -0.177430   
               2013-12-27  False  0.440448        -0.177430   
               2013-11-27  False  0.440448        -0.177430   
               2013-10-27  False  0.440448        -0.177430   
               2013-09-27  False  0.440448        -0.177430   
               2013-08-27  False  0.440448        -0.177430   
               2013-07-27  False  0.440448        -0.177430   
               2013-06-27  False  0.440448        -0.177430   
               2013-05-27  False  0.440448        -0.177430   
...                          ...       ...              ...   
99450          2015-04-10  False -1.141584        -0.302778   
99625          2015-04-07  False  0.856772        -0.806676   
99551          2015-04-21  False  1.606156        -0.757791   
99507          2015-04-08  False -0.891789        -0.755284   
99537          2015-04-08  False -0.225670        -0.639964   
99608          2015-04-29  False -1.224848        -0.400550   
99609          2015-04-09  False -0.475465        -0.804169   
99596          2015-04-27  False  0.357183        -0.762805   
99638          2015-04-10  False  0.940037        -0.804169   
99657          2015-04-20  False  0.357183        -0.251386   
99660          2015-04-13  False  0.024124        -0.792888   
99650          2015-04-15  False  0.773508        -0.756537   
99639          2015-04-17  False -0.808524        -0.602359   
99614          2015-04-16  False  1.106567        -0.637457   
99760          2015-04-27  False -1.474643        -0.177430   
99740          2015-04-24  False -0.308935        -0.794142   
99770          2015-04-28  False  0.024124        -0.553474   
99787          2015-04-15  False  1.439626        -0.792888   
99811          2015-04-15  False  1.273096        -0.794142   
99779          2015-04-17  False -0.641995        -0.177430   
99799          2015-04-15  False -0.808524        -0.736482   
99876          2015-04-17  False -1.308113        -0.549713   
99935          2015-04-25  False -0.725259        -0.164896   
99903          2015-04-28  False  0.440448        -0.666287   
99956          2015-04-24  False  0.606978        -0.683836   
99961          2015-04-27  False -1.141584        -0.177430   
100006         2015-04-22  False -1.141584        -0.804169   
100052         2015-04-23  False  0.606978        -0.805423   
100059         2015-04-24  False  0.024124        -0.177430   
100225         2015-04-29  False -0.891789        -0.805423   

                           number_dependents  experience_in

In [ ]:
test_matrix.head()

In [ ]:
#generate global historical info

In [ ]:
apps.head()

In [ ]:
apps_mini = apps[['table_index','ute_id','app_start_date']]
apps_mini = apps_mini.head(10000)
#apps.groupby('ute_id').table_index.count()
apps_full = apps.copy()

In [ ]:
from datetime import datetime


In [ ]:
#Slow way of generating historial apps (takes 10 min)
limit = pd.Timedelta('1095D') 
prev_apps = []

startTime = datetime.now()

for index,row in apps_full.iterrows():
    curr_id = row['ute_id']
    curr_date = row['app_start_date']
    prev_apps.append(len(apps_full[(apps_full['ute_id']==curr_id)\
                                      & ((curr_date - apps_full['app_start_date']) < limit)
                                      & ((curr_date > apps_full['app_start_date']))]))
print datetime.now() - startTime
apps_full['prev_apps'] = prev_apps

In [ ]:
#Another slow way of generating historial apps (takes 10 min)
limit = pd.Timedelta('1095D') 
prev_apps_dict = {}
user_dict = dict(zip(apps['table_index'], apps_mini['ute_id']))
app_start_dict = dict(zip(apps['table_index'], apps_mini['app_start_date']))

startTime = datetime.now()

for app_id in apps_mini.table_index:
    curr_user = user_dict[app_id]
    curr_date = app_start_dict[app_id]
    prev_apps_dict[app_id] = (len(apps_mini[(apps_mini['ute_id']==curr_user)\
                                     & ((curr_date - apps_mini['app_start_date']) < limit)\
                                     & ((curr_date - apps_mini['app_start_date'])> pd.Timedelta('0D'))]))
print datetime.now() - startTime
apps_mini['prev_apps'] = apps_mini['table_index'].map(prev_apps_dict)

In [ ]:
apps_full.head()

In [ ]:
from utils import feature_utils

In [ ]:
apps_count = apps[['table_index','ute_id','app_start_date']]
apps_count['apps']=1
apps_cumsum = apps_count.groupby(['ute_id','table_index','app_start_date']).sum().groupby(level=[0]).cumsum()

apps_cumsum.head(5)

In [ ]:
apps_cumsum.reset_index(level=[2], inplace = True)
apps_cumsum.rename(columns = {'apps':'apps_cumsum'}, inplace=True)
apps_cumsum.reset_index(level=[0,1], inplace = True)
apps_cumsum['date_type']='app_start_date'
apps_cumsum['prev_apps_cumsum'] = apps_cumsum['apps_cumsum']-1
apps_cumsum.head()

In [ ]:
historical_limit = pd.Timedelta('1460D')

apps_markers = pd.DataFrame(apps.groupby(['ute_id','table_index']).app_start_date.max())
apps_markers['historical_st_date'] = apps_markers['app_start_date']-historical_limit
apps_markers = pd.DataFrame(apps_markers.stack())
apps_markers.reset_index(level=[0,1,2], inplace = True)
apps_markers.rename(columns = {'level_2':'date_type', 0:'date'}, inplace=True)
apps_markers.sort_values(['ute_id','date'], inplace=True)
apps_markers.head()

In [ ]:
apps_historical = pd.merge(apps_markers, apps_cumsum, how='left', left_on= ['ute_id','table_index','date_type','date'], right_on=['ute_id','table_index','date_type','app_start_date']).drop(['apps_cumsum', 'app_start_date'], axis=1)

apps_historical[apps_historical['ute_id']==836]

In [ ]:
apps_historical.fillna(method='bfill', inplace=True)
apps_historical[apps_historical['ute_id']==836]

In [ ]:
apps_historical = pd.pivot_table(apps_historical, values = "prev_apps_cumsum", index = ['ute_id', 'table_index'], columns = 'date_type')

In [ ]:
apps_historical['prev_apps_within_limit'] = apps_historical['app_start_date'] - apps_historical['historical_st_date']

In [ ]:
apps_historical = apps_historical.reset_index(level=[0,1]).drop(['historical_st_date'], axis=1)
#apps_historical[apps_historical['ute_id']==836]

In [ ]:
apps_historical.head()